In [2]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests


In [45]:

# Configuration and setup
API_TOKEN = "apify_api_I76OdSShW712npTQOBUbyGGWmKRBoD4nDRzb"
BASE_URL = "https://api.apify.com/v2/datasets"

headers = {
    "Authorization": f"Bearer {API_TOKEN}"
}

# Function to get all dataset IDs
def get_all_dataset_ids():
    dataset_ids = []
    params = {"limit": 1000, "offset": 0}  # Set limits for pagination
    while True:
        response = requests.get(BASE_URL, headers=headers, params=params)
        data = response.json().get("data", {}).get("items", [])
        
        if not data:
            break  # Stop if there are no more datasets

        # Collect dataset IDs
        dataset_ids.extend([dataset["id"] for dataset in data])
        params["offset"] += len(data)  # Move to the next page
    return dataset_ids

# Retrieve all tweets from the datasets
all_tweets = []
dataset_ids = get_all_dataset_ids() 

for dataset_id in dataset_ids:
    url = f"{BASE_URL}/{dataset_id}/items?format=json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        tweets = response.json()
        all_tweets.extend(tweets)  

# Convert tweets to DataFrame
tweets_df = pd.json_normalize(all_tweets)

# Convert text column if needed and handle missing or non-standard fields
tweets_df['text'] = tweets_df.get('fullText', '').astype(str)

# Remove duplicates based on 'text' or another column (e.g., 'id')
tweets_df = tweets_df.drop_duplicates(subset=['text'])

# Save to Excel
tweets_df.to_excel("unique_tweets.xlsx", index=False)
print("Data has been saved to 'unique_tweets.xlsx'.")

Data has been saved to 'unique_tweets.xlsx'.


### Get named and unnamed datasets

In [2]:
import requests
import pandas as pd

# Configuration and setup
API_TOKEN = "apify_api_dWKOEBf1gyHNFa9eHJY1AaaXHsysA23GgCFT"
BASE_URL = "https://api.apify.com/v2/datasets"

headers = {
    "Authorization": f"Bearer {API_TOKEN}"
}

# Function to get all dataset IDs (including unnamed ones)
def get_all_dataset_ids():
    dataset_ids = []
    params = {"limit": 1000, "offset": 0, "unnamed": 1}  # Add 'unnamed=1' to include unnamed datasets
    while True:
        response = requests.get(BASE_URL, headers=headers, params=params)
        
        if response.status_code != 200:
            print("Error fetching datasets:", response.status_code, response.text)
            break  # Exit if there's an error in the response
        
        data = response.json().get("data", {}).get("items", [])
        
        if not data:
            break  # Stop if there are no more datasets

        # Collect all dataset IDs, named or unnamed
        dataset_ids.extend([dataset["id"] for dataset in data])
        params["offset"] += len(data)  # Move to the next page
        
    return dataset_ids

# Retrieve all tweets from the datasets
all_tweets = []
dataset_ids = get_all_dataset_ids()

# Diagnostic print to verify dataset retrieval
print(f"Total datasets retrieved: {len(dataset_ids)}")

for dataset_id in dataset_ids:
    url = f"{BASE_URL}/{dataset_id}/items?format=json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        tweets = response.json()
        all_tweets.extend(tweets)
    else:
        print(f"Failed to retrieve dataset with ID {dataset_id}: {response.status_code}")

# Convert tweets to DataFrame
tweets_df = pd.json_normalize(all_tweets)

# Convert text column if needed and handle missing or non-standard fields
tweets_df['text'] = tweets_df.get('fullText', '').astype(str)

# Remove duplicates based on 'text' or another column (e.g., 'id')
tweets_df = tweets_df.drop_duplicates(subset=['text']) 

# Save to Excel
tweets_df.to_excel("unique_tweets_2023.xlsx", index=False)
print("Data has been saved to 'unique_tweets_extended2.xlsx'.")


Total datasets retrieved: 1281
Data has been saved to 'unique_tweets_extended2.xlsx'.
